In [6]:
import json
import pandas as pd

In [7]:
def get_meta_prompt(file_path): #Plug in path to csv
    df = pd.read_csv(file_path)
    questions = df["Model Questions"]
    meta_prompt = """Given the following group of statements, classify a user statement into most similar group, if there are no matches, output 0 for question group, use the "classify" function provided:\nList of groups: """
    for i, question in enumerate(questions):
        group = str(i) + ": " + question
        meta_prompt += group

In [8]:
import numpy as np
import json
import slack
import openai
import os
from pathlib import Path
from dotenv import load_dotenv
from flask import Flask
from slackeventsapi import SlackEventAdapter
import pandas as pd
from doormanFailure import *

global logs
def value_exists_in_json(file_path, value):
    data = load_json_from_file(file_path)
    print(data)
    #return value == data

def write_json_to_file(data, file_path):
    with open(file_path, 'w') as f:
        json.dump(data, f)

def load_json_from_file(file_path):

    with open(file_path, 'r') as f:
        data = json.load(f)
    return data['logs']

def append_log(logs, t_id, role, content):
    for log in logs:
        if (log['t_id'] == t_id):
            log['thread_log'].append({"role": role, "content": cLastontent})
            return logs
    new_log = {
        "t_id": t_id,
        "thread_log": [
            {"role": role, "content": content}
        ],
    }
    logs.append(new_log)
    return logs
def load_log(logs, t_id):
    for log in logs:
        if (log['t_id'] == t_id):
            return log['thread_log']
    return []

def doorman_v0(t_id, prompt, answers):
    global logs
    # Function params, mess around with these to experiment with model behvior
    null_response = "I'm sorry, could you provide more clarifying information?"
    max_tokens = 100
    temperature = 0.8
    with open('llmlads_prompt.txt','r') as file:
        meta_prompt = file.read()
    # Load thread log if exists
    system_prompt = np.array([{"role": "system", "content": meta_prompt}])
    user_prompt = np.array([{"role": "user", "content": prompt}])
    thread_log = np.array(load_log(logs, t_id))
    messages = np.concatenate((system_prompt, thread_log, user_prompt)).tolist()
    #print(messages)
    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo-16k",
        messages = messages,
        functions = [
            {
                "name": "classify",
                "description": "classify a user statement into one of the statement groups using a json format, output 0 if there are no matches",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "group": {
                            "type": "number",
                            "description": "index of group that the statement most resembles",
                        },
                    },
                },
                "required": ["group"],
            },
        ],
        function_call="auto",
        max_tokens = 100,
        temperature = 0.8,
    )
    try:
        if (response["choices"][0]["message"]["function_call"]["arguments"] != "{}"):
            index = int(response["choices"][0]["message"]["function_call"]["arguments"][13:-2])
            answer = answers[index - 1]
            logs = append_log(logs, t_id, "user", prompt)
            logs = append_log(logs, t_id, "assistant", answer)
            return index
        else:
            print(logs)
            logs = append_log(logs, t_id, "user", prompt)
            print(logs)
            logs = append_log(logs, t_id, "assistant", null_response)
            print("Null response returned")
            return 0
    except:
        logs = append_log(logs, t_id, "user", prompt)
        logs = append_log(logs, t_id, "assistant", null_response)
        print("<dev flag> Function wasn't even called lol")
        return 0
    
################################################
#### surround this code with __name__ block ####
################################################
"""
if __name__ == "__main__":
    df = pd.read_csv('QandA_list.csv', encoding='utf-8')
    answers = df["Question answers"].tolist()
    env_path = Path('.')/'.env'
    load_dotenv(dotenv_path=env_path)
    openai.api_key = token=os.environ['CHAT_TOKEN']
    try:
        logs = load_json_from_file("doorman_log.json")
        while(True):
            t_id = int(input("Thread ID: "))
            prompt = input("User prompt: ")
            index = doorman_v0(logs, t_id, prompt, answers)
            if index == 0:
                print("Not found")
            else:
                print("Found")
                print(answers[index-1])
            write_json_to_file({"logs" :logs}, "doorman_log.json")
    except:
        if os.path.isfile("doorman_log.json"):
            print("<dev flag> something went wrong, but doorman_log.json exists")
        else:
            print("<dev flag> doorman log not found, making empty file called doorman_log.json")
            logs = {"logs" : []}
            write_json_to_file(logs, "doorman_log.json")
"""
################################################
################################################
################################################
def doormanWMemory(input,thread_id):
    global logs
    df = pd.read_csv('QandA_list.csv', encoding='utf-8')
    answers = df["Question answers"].tolist()
    try:
        logs = load_json_from_file("doorman_log.json")
        t_id = thread_id
        prompt = input
        index = doorman_v0(t_id, prompt, answers)
        print(logs)
        if index == 0:
            print("Not found")
            write_json_to_file({"logs" :logs}, "doorman_log.json")
            return(doormanFailure())
        else:
            print("Found")
            write_json_to_file({"logs" :logs}, "doorman_log.json")
            return(answers[index-1])
    except Exception as e:
        if os.path.isfile("doorman_log.json"):
            print("<dev flag> something went wrong, but doorman_log.json exists")
            print(e)
        else:
            print("<dev flag> doorman log not found, making empty file called doorman_log.json")
            logs = {"logs" : []}
            write_json_to_file(logs, "doorman_log.json")
            return doormanWMemory(input,thread_id)

def checkFor(data):
    return value_exists_in_json("doorman_log.json",data)

ModuleNotFoundError: No module named 'doormanFailure'

In [48]:
from datetime import datetime
import pandas as pd

def formatted_time():
    now =                       datetime.now()
    formatted_time =            now.strftime('%H-%M-%d-%m-%Y')
    return formatted_time
def append_question(
                    question_categories = "", # title of question set
                    question_answers= "", # answer to be given by Doorman
                    model_questions= "", # question set to be plugged into doorman
                    keywords = [""], # keywords associated with question set (optional)
                    filepath = "QandA_list.csv",): # path to the csv
    data = pd.read_csv(filepath)
    print(data.keys())
    new_row = {'Question Categories': question_categories,
               'Question answers': question_answers,
               'Model Questions': model_questions,
               'Keywords': keywords
              }
    data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)
    return data
    """
    except:
        print("File not found, please fix filepath to csv")
        return None
    """
# To use this function, first create a ./backup/ directory where you are running the script
def save_csv(data, filepath = "QandA_list.csv", backup = True):
    old_data = pd.read_csv(filepath)
    data.to_csv(filepath, index=False)
    if backup == True:
        backup_path = "./backup/" + formatted_time() + "_QandA_list.csv"
        old_data.to_csv(backup_path)
def remove_question(data, index):
    return data.drop([data.index[index]])

# todo: add these functions after Mateo figures out the modal stuff
def edit_question():
    pass

if __name__ == "__main__":
    filepath = 'QandA_list.csv'
    data = pd.read_csv(filepath)
    print(data.keys())
    """
    data = append_question(
        question_categories = "Add TA to iCollege",
        question_answers = "Fill out a request form: https://gsutech.service-now.com/sp?id=sc_cat_item&sys_id=6dbd8365db501340601b502bdc9619dc",
        model_questions = (
            "How to add a TA with admin rights to my class?"
            "Add a TA with full access to my class on icollege?"
            "How to grant admin rights to my TA so they can manage the class?"
            "How can I enable my TA to enter grades?"
        ),
        keywords = ["TA", "teaching assistant", "iCollege access"]
    )
    
    """
    data = remove_question(data, 28)
    save_csv(data)

Index(['Question Categories', 'Question answers', 'Model Questions',
       'Keywords'],
      dtype='object')


Index(['Question Categories', 'Question answers', 'Model Questions',
       'Keywords'],
      dtype='object')
